## $\textbf{Inferring Musical Time Signature via Apple Watch Logs of Conductor Wrist Motion}$

$\textbf{Author:}\text{ Ryan Burns}$

$\text{When conducting a chamber group or orchestra, a musical conductor uses a designated baton pattern for each possible time signature a musical}$
$\text{score could be written in. Suppose someone were to wear an Apple Watch on their right wrist while conducting, (assume watch IMU data is}$
$\text{logged at 100 Hz). It is not unreasonable to suspect that we could train a machine learning model to predict the time signature corresponding }$
$\text{to particular wrist motion patterns. A small amount of data is studied to prove this concept using }\textit{long short-term memory (LSTM)}\text{ networks.}$
$\text{For a great overview of LSTM and other recurrent units, see: }\textit{https://colah.github.io/posts/2015-08-Understanding-LSTMs/}\text{. While intended }$
$\text{for sequential data (e.g., the quasi-cyclostationary wrist motions of a conductor), we also admit that this is a small amount of training and }$
$\text{validation data, so the generalization performance of these models across a variety of conductors, time signatures, and tempos could prove}$
$\text{quite poor. With generalization performance being a concern, we a couple techniques to hopefully mitigate some amount of overfitting. }$

$\text{We add Gaussian noise to the input signals to randomly vary input to the network, in addition to applying random dropout to a subset of}$
$\text{the connections within the network at a fixed probability. The model is trained on a subset of data and validated on the remaining data.}$

$\text{This specific deep learning architecture is comprised of a single convolutional layer fed to a dense hyperbolic tangent layer for feature,}$
$\text{extraction preceding the long short-term memory layer. The LSTM units make use of both types of dropout available (ordinary and recurrent)}$
$\text{as measures to reduce overfitting. A multidimensional sliding window is applied to the accelerometer, gyroscope, and IMU-fused watch}$
$\text{orientation quaternion at the input layer of the network.}$

$\textbf{Motion Class Labels:}$

$\text{All class labels are defined for a right-handed user. An orchestral conductor varies their baton pattern according to 1 of 4 possible states, }$
$\text{comprised of 3 time signature classes and a resting class (i.e., cessation of baton motion).}$

$\text{0 }\leftrightarrow [1\enspace0\enspace0\enspace0]\leftrightarrow \text{REST}\Longrightarrow\text{conductor has ceased baton motion (no conducting)}$

$\text{1 }\leftrightarrow [0\enspace1\enspace0\enspace0]\leftrightarrow {2/4}\Longrightarrow\text{conductor proceeds with baton motion consistent with a }\mathbf{\genfrac{}{}{0pt}{}{2}{4}}\text{ time signature}$

$\text{2 }\leftrightarrow [0\enspace0\enspace1\enspace0]\leftrightarrow {3/4}\Longrightarrow\text{conductor proceeds with baton motion consistent with a }\mathbf{\genfrac{}{}{0pt}{}{3}{4}}\text{ time signature}$

$\text{3 }\leftrightarrow [0\enspace0\enspace0\enspace1]\leftrightarrow {4/4}\Longrightarrow\text{conductor proceeds with baton motion consistent with a }\mathbf{\genfrac{}{}{0pt}{}{4}{4}}\text{ time signature}$

$\text{For more information on musical time signatures, visit: }\textit{https://en.wikipedia.org/wiki/Time_signature}.$

![title](baton_motion.png)

$\text{The baton patterns for each time signature of interest are depicted diagramatically above. We assume a tech enthusiast conductor who desires}$
$\text{an Apple Watch app/experience for automatic discrimination between the 3 time signatures above, in addition to a catch-all }\textit{at-rest}\text{ state. We }$
$\text{also assume that this conductor would like automatic time-signature inference to be as tempo-agnostic as possible. Be it }\textit{largo}\text{ or }\textit{prestissimo}\text{,}$
$\text{we assume that the tempo of the musical composition of question would not fool the ideal baton pattern classifier. As such, while the amount}$
$\text{of data collected for this analysis is still limited in its size and diversity (i.e., we can assume overfit models), efforts have been made during}$
$\text{data collection to vary the tempo across each time signature's constituent wrist motion observations. The duration (in seconds or measures)}$
$\text{of each time signature's wrist motion subsequence is also varied during collection. We create an aggregated dataset of independent concatenated}$
$\text{collects for supervised learning in the code below.}$

$\textbf{Note On Labeling:}$

$\text{SensorLog labels are recorded in real time using the app's class label buttons for a streaming iPhone. This iPhone logs data concurrently with}$
$\text{an Apple Watch, which also reports its own class labels. Since toggling of the Apple Watch's class labels using the SensorLog UI on the watch }$
$\text{face would interfere with data collection of wrist motion, we use the }\textit{iPhone}\text{ to log wrist motion labels. By time-aligning the iPhone and Apple}$
$\text{Watch streams below (i.e., using POSIX timestamps), we can readily provide wrist motion labels for the Apple Watch motion signals without}$
$\text{interfering with their trajectory as just described. In short, real-time motion labeling is available through dual stream of Apple Watch }$
$\text{iPhone data, where the former provides the motion observations of interest and the latter provides a mechanism for real-time motion labeling.}$

### $\textbf{Import Packages}$

In [1]:
import pandas as pd
from os import getcwd, environ;
from itertools import product as iter_prod;
from matplotlib import pyplot as plt;
from mpl_toolkits.axes_grid1 import make_axes_locatable
from numpy import array, hstack, argmax, ones, zeros, log10;
from numpy import logical_or, logical_not, expand_dims, flip;
from numpy import  abs, arange, shape, newaxis, sum, flipud;
from numpy import nan_to_num;
from scipy.signal import parzen;

from tensorflow.keras.models import Sequential, Model;
from tensorflow.keras.optimizers import RMSprop;
from tensorflow.keras.metrics import CategoricalAccuracy;
from tensorflow.keras.metrics import Precision, Recall, AUC;
from tensorflow.keras.layers import Dropout, LSTM, Dense;
from tensorflow.keras.layers import Activation, Reshape, Conv1D;
from tensorflow.keras.layers import Input, GaussianNoise;
from sklearn.metrics import confusion_matrix;

from SensorLogUtils import convert_iPhone_units;

environ['KMP_DUPLICATE_LIB_OK']='True'

### $\textbf{Class Label Definitions}$

In [2]:
# Ordinal motion class labels
class_table = pd.DataFrame({
    'REST': {
        'id': 'REST', 
        'description': 'no conducting / baton pattern',
        '1-hot label': [1,0,0,0],
        'ordinal label': 0
    },
    '2/4': {
        'id': '2/4', 
        'description': 'conducting pattern for a 2/4 time signature',
        '1-hot label': [0,1,0,0],
        'ordinal label': 1
    },
    '3/4': {
        'id': '3/4', 
        'description':'conducting pattern for a 3/4 time signature',
        '1-hot label': [0,0,1,0],
        'ordinal label': 2
    },
    '4/4': {
        'id': '4/4', 
        'description':'conducting pattern for a 4/4 time signature',
        '1-hot label': [0,0,0,1],
        'ordinal label': 3
    }
});

# Number of classes
C = 4;

# Print class table
class_table

,REST,2/4,3/4,4/4
id,REST,2/4,3/4,4/4
description,no conducting / baton pattern,conducting pattern for a 2/4 time signature,conducting pattern for a 3/4 time signature,conducting pattern for a 4/4 time signature
1-hot label,"[1, 0, 0, 0]","[0, 1, 0, 0]","[0, 0, 1, 0]","[0, 0, 0, 1]"
ordinal label,0,1,2,3


### $\textbf{Specify & Load Dataset}$

In [3]:
################
# Specify data #
################

# Specify local data storage path
data_path = getcwd() + '/data';

# Collect file string ID
collect_IDs = [
    'time_signatures_collect1',
    'time_signatures_collect2'
];

# Build string filename corresponding to collect_ID
collect_files = [
    (data_path + '/labeled_' + collect_ID + '_appleWatch.csv')
for collect_ID in collect_IDs];

#############
# Load data #
#############

# Load and concatenate dataframes for all
# labeled collects in list collect_IDs
df = pd.concat([pd.read_csv(f, 
    error_bad_lines=False,
    warn_bad_lines=False)
    for f in collect_files
],axis=0,ignore_index=True);

# Data fields (column headers)
fields = [fd for fd in df];

# Length of dataframe
N = len(df); # (samples)

### $\textbf{Select IMU Data & 1-Hot Class Labels}$

In [4]:
# Select input signals
x = nan_to_num(array(df[[
    'accelerometerAccelerationX(m/s^2)',
    'accelerometerAccelerationY(m/s^2)',
    'accelerometerAccelerationZ(m/s^2)',
    'motionRotationRateX(rad/s)',
    'motionRotationRateY(rad/s)',
    'motionRotationRateZ(rad/s)',
    'motionQuaternionW(R)',
    'motionQuaternionX(R)',
    'motionQuaternionY(R)',
    'motionQuaternionZ(R)'
]]));

# Define 1-hot labels
y = nan_to_num(array(df[[
    
    # List all class headers in table above
    class_name for class_name in class_table
]]));

### $\textbf{Specify Sliding Observation Window Parameters}$

In [5]:
# Number of raw sensor samples (N)
# and their dimensionality (D)
N, D = shape(x);

# Sliding obs. window length
M = 128; # (samples)

# Step size of obs. window
m = 10;  # (samples)

# Linear grid for sliding observation
# window reflecting window length & step
# size specified above, mapped to raw
# sensor samples n = M, M + m, M + 2m, ...
obs_idx = [n for n in range(M,N,m)];

# Define Parzen window to weight observations
w = expand_dims(parzen(M),axis=1);

### $\textbf{Define Observations & Class Labels}$

In [6]:
# Windowed observation sequence input to model
X = array([w * x[(n - M):n,:] for n in obs_idx]);

# Ground truth 1-hot window-resolution class labels
Y = array([
    [1 if c == argmax(sum(w * y[(n - M):n,:],0)) 
    else 0 for c in range(C)] for n in obs_idx]);

### $\textbf{Training Parameters}$

In [7]:
# Fraction of dataset to hold 
# out for model validation
pct_validation = 0.25;

# Total number of obs.
N_obs = len(X);

# Number of training epochs
N_epoch = 5000;

# Number of batches
N_batch = int(N_obs / 1);

### $\textbf{Define Machine Learning Model}$

In [8]:
#####################################################
# Define model architecture w/ Keras functional API #
#####################################################

# Input layer - M x D observation window
input_layer = Input((M,D));

# Add Gaussian noise to M x D input
h0 = GaussianNoise(5e-3)(input_layer);

# Convolutional layer, rectified linear unit activation
h1 = Conv1D(M,M,activation='relu')(h0);

# Dense hyperbolic tangent activation layer
h2 = Dense(M,activation='tanh')(h1);

# Long short-term memory layer
h3 = LSTM(2 * M,activation='tanh',
    dropout=0.5,
    recurrent_dropout=0.5)(h2);

# Output layer - C x 1 softmax class activation 
output_layer = Dense(C,activation='softmax')(h3);

# Set up Keras Model() instance
model = Model(
    inputs=input_layer,  # Model inputs
    outputs=output_layer # Model outputs
);

###########################
# Define loss & optimizer #
###########################

# Set RMSprop optimization for 
# speed-of-convergence purposes
opt = RMSprop(
    learning_rate=0.001,
    rho=0.9,
    momentum=0.001,
    epsilon=1e-07,
    name="RMSprop"
);

# Model compilation, using categorical
# cross-entropy error w/ RMSprop
model.compile(
    
    # Error/loss function
    loss='categorical_crossentropy', 
    
    # Use RMSprop
    optimizer=opt,
    
    # List metrics here
    metrics=[
        CategoricalAccuracy(),
        AUC(),
        Precision(),
        Recall()
    ]
);

# Print a summary table
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 10)]         0         
_________________________________________________________________
gaussian_noise (GaussianNois (None, 128, 10)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 1, 128)            163968    
_________________________________________________________________
dense (Dense)                (None, 1, 128)            16512     
_________________________________________________________________
lstm (LSTM)                  (None, 256)               394240    
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 1028      
Total params: 575,748
Trainable params: 575,748
Non-trainable params: 0
_______________________________________________________

### $\textbf{Train Machine Learning Model}$

In [ ]:
# Model training
model.fit(
    
    # Input dataset
    x=X, # windowed observations
    y=Y, # 1-hot motion class labels
    
    # Batch size
    batch_size=N_batch,
    
    # Number of training epochs
    epochs=N_epoch,
    
    # Print progress
    verbose=1,
    
    # Set aside fraction for validation
    validation_split=pct_validation,
    
    # False for time series
    shuffle=False,
    
    # Other misc params
    sample_weight=None,
    validation_freq=1,
    max_queue_size=10,
    
    # Parallelize job across 2 workers
    workers=2,
    use_multiprocessing=True
);

Train on 6156 samples, validate on 2052 samples
Epoch 1/5000
6156/6156 [==============================] - 5s 835us/sample - loss: 1.3873 - categorical_accuracy: 0.2437 - auc: 0.5079 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.2486 - val_categorical_accuracy: 0.3952 - val_auc: 0.7334 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/5000
6156/6156 [==============================] - 1s 160us/sample - loss: 1.3041 - categorical_accuracy: 0.3455 - auc: 0.6630 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.1998 - val_categorical_accuracy: 0.4727 - val_auc: 0.7675 - val_precision: 0.9773 - val_recall: 0.0210
Epoch 3/5000
6156/6156 [==============================] - 1s 148us/sample - loss: 1.2203 - categorical_accuracy: 0.4683 - auc: 0.7421 - precision: 0.8750 - recall: 0.0182 - val_loss: 1.0531 - val_categorical_accuracy: 0.5336 - val_auc: 0.8251 - val_precision: 0.8161 - val_recall: 0.2378
Epoch 4/5000
6156/6156 [==============================] - 1s 150

Epoch 30/5000
6156/6156 [==============================] - 1s 145us/sample - loss: 0.3684 - categorical_accuracy: 0.8613 - auc: 0.9759 - precision: 0.8778 - recall: 0.8390 - val_loss: 0.3598 - val_categorical_accuracy: 0.8962 - val_auc: 0.9795 - val_precision: 0.9151 - val_recall: 0.8670
Epoch 31/5000
6156/6156 [==============================] - 1s 148us/sample - loss: 0.3471 - categorical_accuracy: 0.8765 - auc: 0.9802 - precision: 0.8915 - recall: 0.8611 - val_loss: 0.4036 - val_categorical_accuracy: 0.8402 - val_auc: 0.9709 - val_precision: 0.8682 - val_recall: 0.8187
Epoch 32/5000
6156/6156 [==============================] - 1s 142us/sample - loss: 0.3363 - categorical_accuracy: 0.8713 - auc: 0.9801 - precision: 0.8882 - recall: 0.8532 - val_loss: 0.3710 - val_categorical_accuracy: 0.8655 - val_auc: 0.9755 - val_precision: 0.8824 - val_recall: 0.8523
Epoch 33/5000
6156/6156 [==============================] - 1s 144us/sample - loss: 0.3282 - categorical_accuracy: 0.8790 - auc: 0.981

Epoch 59/5000
6156/6156 [==============================] - 1s 145us/sample - loss: 0.3381 - categorical_accuracy: 0.8635 - auc: 0.9799 - precision: 0.8724 - recall: 0.8528 - val_loss: 0.3236 - val_categorical_accuracy: 0.9103 - val_auc: 0.9801 - val_precision: 0.9168 - val_recall: 0.9025
Epoch 60/5000
6156/6156 [==============================] - 1s 146us/sample - loss: 0.2996 - categorical_accuracy: 0.8790 - auc: 0.9837 - precision: 0.8865 - recall: 0.8720 - val_loss: 0.4648 - val_categorical_accuracy: 0.8207 - val_auc: 0.9631 - val_precision: 0.8354 - val_recall: 0.8090
Epoch 61/5000
6156/6156 [==============================] - 1s 147us/sample - loss: 0.3128 - categorical_accuracy: 0.8874 - auc: 0.9811 - precision: 0.8951 - recall: 0.8764 - val_loss: 0.3150 - val_categorical_accuracy: 0.9035 - val_auc: 0.9806 - val_precision: 0.9110 - val_recall: 0.8977
Epoch 62/5000
6156/6156 [==============================] - 1s 146us/sample - loss: 0.2404 - categorical_accuracy: 0.9185 - auc: 0.989

Epoch 88/5000
6156/6156 [==============================] - 1s 147us/sample - loss: 0.2164 - categorical_accuracy: 0.9116 - auc: 0.9913 - precision: 0.9159 - recall: 0.9059 - val_loss: 0.2710 - val_categorical_accuracy: 0.9215 - val_auc: 0.9848 - val_precision: 0.9293 - val_recall: 0.9162
Epoch 89/5000
6156/6156 [==============================] - 1s 148us/sample - loss: 0.1927 - categorical_accuracy: 0.9228 - auc: 0.9928 - precision: 0.9281 - recall: 0.9189 - val_loss: 0.3436 - val_categorical_accuracy: 0.8689 - val_auc: 0.9799 - val_precision: 0.8754 - val_recall: 0.8665
Epoch 90/5000
6156/6156 [==============================] - 1s 151us/sample - loss: 0.1830 - categorical_accuracy: 0.9300 - auc: 0.9935 - precision: 0.9330 - recall: 0.9269 - val_loss: 0.2658 - val_categorical_accuracy: 0.9264 - val_auc: 0.9854 - val_precision: 0.9325 - val_recall: 0.9230
Epoch 91/5000
6156/6156 [==============================] - 1s 149us/sample - loss: 0.1761 - categorical_accuracy: 0.9331 - auc: 0.994

Epoch 117/5000
6156/6156 [==============================] - 1s 147us/sample - loss: 0.1358 - categorical_accuracy: 0.9576 - auc: 0.9964 - precision: 0.9614 - recall: 0.9552 - val_loss: 0.2925 - val_categorical_accuracy: 0.9064 - val_auc: 0.9836 - val_precision: 0.9082 - val_recall: 0.9011
Epoch 118/5000
6156/6156 [==============================] - 1s 158us/sample - loss: 0.1312 - categorical_accuracy: 0.9534 - auc: 0.9969 - precision: 0.9586 - recall: 0.9506 - val_loss: 0.2649 - val_categorical_accuracy: 0.9274 - val_auc: 0.9850 - val_precision: 0.9312 - val_recall: 0.9240
Epoch 119/5000
6156/6156 [==============================] - 1s 158us/sample - loss: 0.1279 - categorical_accuracy: 0.9550 - auc: 0.9968 - precision: 0.9583 - recall: 0.9516 - val_loss: 0.3159 - val_categorical_accuracy: 0.8977 - val_auc: 0.9822 - val_precision: 0.9015 - val_recall: 0.8962
Epoch 120/5000
6156/6156 [==============================] - 1s 150us/sample - loss: 0.1329 - categorical_accuracy: 0.9511 - auc: 0

...

Epoch 3398/5000
6156/6156 [==============================] - 1s 164us/sample - loss: 2.4311e-04 - categorical_accuracy: 1.0000 - auc: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.5224 - val_categorical_accuracy: 0.9293 - val_auc: 0.9691 - val_precision: 0.9293 - val_recall: 0.9288
Epoch 3399/5000
6156/6156 [==============================] - 1s 162us/sample - loss: 1.8563e-04 - categorical_accuracy: 1.0000 - auc: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.5232 - val_categorical_accuracy: 0.9293 - val_auc: 0.9694 - val_precision: 0.9293 - val_recall: 0.9288
Epoch 3400/5000
6156/6156 [==============================] - 1s 158us/sample - loss: 2.5272e-04 - categorical_accuracy: 1.0000 - auc: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.5234 - val_categorical_accuracy: 0.9298 - val_auc: 0.9697 - val_precision: 0.9302 - val_recall: 0.9293
Epoch 3401/5000
6156/6156 [==============================] - 1s 238us/sample - loss: 2.3821e-04 - categorical_accur

Epoch 3426/5000
6156/6156 [==============================] - 3s 534us/sample - loss: 2.2595e-04 - categorical_accuracy: 1.0000 - auc: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.5306 - val_categorical_accuracy: 0.9274 - val_auc: 0.9691 - val_precision: 0.9283 - val_recall: 0.9274
Epoch 3427/5000
6156/6156 [==============================] - 3s 523us/sample - loss: 2.4484e-04 - categorical_accuracy: 1.0000 - auc: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.5311 - val_categorical_accuracy: 0.9274 - val_auc: 0.9691 - val_precision: 0.9283 - val_recall: 0.9274
Epoch 3428/5000
6156/6156 [==============================] - 3s 526us/sample - loss: 2.8096e-04 - categorical_accuracy: 1.0000 - auc: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.5366 - val_categorical_accuracy: 0.9279 - val_auc: 0.9691 - val_precision: 0.9288 - val_recall: 0.9279
Epoch 3429/5000
6156/6156 [==============================] - 3s 528us/sample - loss: 2.1936e-04 - categorical_accur

Epoch 3454/5000
6156/6156 [==============================] - 3s 537us/sample - loss: 2.1894e-04 - categorical_accuracy: 1.0000 - auc: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.5385 - val_categorical_accuracy: 0.9274 - val_auc: 0.9692 - val_precision: 0.9278 - val_recall: 0.9274
Epoch 3455/5000
6156/6156 [==============================] - 3s 529us/sample - loss: 2.8661e-04 - categorical_accuracy: 1.0000 - auc: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.5405 - val_categorical_accuracy: 0.9269 - val_auc: 0.9688 - val_precision: 0.9269 - val_recall: 0.9269
Epoch 3456/5000
6156/6156 [==============================] - 3s 525us/sample - loss: 2.5497e-04 - categorical_accuracy: 1.0000 - auc: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.5377 - val_categorical_accuracy: 0.9279 - val_auc: 0.9692 - val_precision: 0.9278 - val_recall: 0.9274
Epoch 3457/5000
6156/6156 [==============================] - 3s 522us/sample - loss: 2.1898e-04 - categorical_accur

Epoch 3482/5000
6156/6156 [==============================] - 3s 527us/sample - loss: 0.0163 - categorical_accuracy: 0.9948 - auc: 0.9995 - precision: 0.9948 - recall: 0.9948 - val_loss: 0.7348 - val_categorical_accuracy: 0.8996 - val_auc: 0.9549 - val_precision: 0.9000 - val_recall: 0.8996
Epoch 3483/5000
6156/6156 [==============================] - 3s 538us/sample - loss: 0.0234 - categorical_accuracy: 0.9935 - auc: 0.9994 - precision: 0.9935 - recall: 0.9935 - val_loss: 0.4467 - val_categorical_accuracy: 0.9386 - val_auc: 0.9760 - val_precision: 0.9400 - val_recall: 0.9386
Epoch 3484/5000
6156/6156 [==============================] - 3s 545us/sample - loss: 0.0186 - categorical_accuracy: 0.9943 - auc: 0.9995 - precision: 0.9943 - recall: 0.9943 - val_loss: 0.6909 - val_categorical_accuracy: 0.9089 - val_auc: 0.9582 - val_precision: 0.9093 - val_recall: 0.9089
Epoch 3485/5000
6156/6156 [==============================] - 3s 523us/sample - loss: 0.0188 - categorical_accuracy: 0.9945 - au

Epoch 3510/5000
6156/6156 [==============================] - 7202s 1s/sample - loss: 2.4553e-04 - categorical_accuracy: 1.0000 - auc: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.5262 - val_categorical_accuracy: 0.9250 - val_auc: 0.9704 - val_precision: 0.9254 - val_recall: 0.9245
Epoch 3511/5000
6156/6156 [==============================] - 1s 134us/sample - loss: 2.4080e-04 - categorical_accuracy: 1.0000 - auc: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.5252 - val_categorical_accuracy: 0.9259 - val_auc: 0.9704 - val_precision: 0.9263 - val_recall: 0.9254
Epoch 3512/5000
6156/6156 [==============================] - 1s 135us/sample - loss: 3.3477e-04 - categorical_accuracy: 1.0000 - auc: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.5292 - val_categorical_accuracy: 0.9259 - val_auc: 0.9701 - val_precision: 0.9259 - val_recall: 0.9254
Epoch 3513/5000
6156/6156 [==============================] - 1s 134us/sample - loss: 2.3548e-04 - categorical_accur

### $\textbf{Metrics vs. Training Epoch}$

In [ ]:
%matplotlib notebook

# Text legend labels for training set curves
train_set_lbl = ('Training Set (' + 
    str(int(100 * (1 - pct_validation))) + '%)');
    
# Text legend labels for validation set curves
val_set_lbl = ('Training Set (' + 
    str(int(100 * (1 - pct_validation))) + '%)');

# New figure
loss_fig = plt.figure(figsize=(9.9,12));

#############
# Loss plot #
#############
ax1 = plt.subplot(5,1,1);

# Add grid to axes
plt.grid(color='k',alpha=0.25);

# Linear grid of training epochs
epochs = arange(0,N_epoch);

# Cross-entropy error (dB) - training set
plt.plot(epochs,10 * log10(
    model.history.history['loss']),
    '.-',c='g',alpha=0.5,
    label=train_set_lbl);

# Cross-entropy error (dB) - validation set
plt.plot(epochs,10 * log10(
    model.history.history['val_loss']),
    '.-',c='k',alpha=0.5,
    label=val_set_lbl);

# Set x-axis limits
plt.xlim([epochs[0],epochs[-1]]);

# Title
plt.title(r'Cross-Entropy Error vs. Training Epoch $\varepsilon$',
          fontsize=16,weight='bold');

# x-axis label
plt.xlabel(r'Epoch $\varepsilon$',fontsize=14);

# y-axis label
plt.ylabel(r'Error${}_{\varepsilon}$ (dB)',fontsize=14);

# Legend
plt.legend();

#################
# Accuracy plot #
#################
plt.subplot(5,1,2,sharex=ax1);

# Add grid to axes
plt.grid(color='k',alpha=0.25);

# Cross-entropy error (dB) - training set
plt.plot(epochs,model.history.history['categorical_accuracy'],
    '.-',c='g',alpha=0.5,label=train_set_lbl);

# Cross-entropy error (dB) - validation set
plt.plot(epochs,model.history.history['val_categorical_accuracy'],
    '.-',c='k',alpha=0.5,label=val_set_lbl);

# Set x-axis limits
plt.xlim([epochs[0],epochs[-1]]);

# Title
plt.title(r'Categorical Accuracy vs. Training Epoch $\varepsilon$',
          fontsize=16,weight='bold');

# x-axis label
plt.xlabel(r'Epoch $\varepsilon$',fontsize=14);

# y-axis label
plt.ylabel(r'Accuracy${}_{\varepsilon}$ ($\times100\%$)',fontsize=14);

# Legend
plt.legend();

##################
# Precision plot #
##################
plt.subplot(5,1,3,sharex=ax1);

# Add grid to axes
plt.grid(color='k',alpha=0.25);

# Cross-entropy error (dB) - training set
plt.plot(epochs,model.history.history['precision'],
    '.-',c='g',alpha=0.5,label=train_set_lbl);

# Cross-entropy error (dB) - validation set
plt.plot(epochs,model.history.history['val_precision'],
    '.-',c='k',alpha=0.5,label=val_set_lbl);

# Set x-axis limits
plt.xlim([epochs[0],epochs[-1]]); 

# x-axis label
plt.xlabel(r'Epoch $\varepsilon$',fontsize=14);

# y-axis label
plt.ylabel(r'Precision${}_{\varepsilon}$',fontsize=14);

# Legend
plt.legend();

###############
# Recall plot #
###############
plt.subplot(5,1,4,sharex=ax1);

# Add grid to axes
plt.grid(color='k',alpha=0.25);

# Cross-entropy error (dB) - training set
plt.plot(epochs,model.history.history['recall'],
    '.-',c='g',alpha=0.5,label=train_set_lbl);

# Cross-entropy error (dB) - validation set
plt.plot(epochs,model.history.history['val_recall'],
    '.-',c='k',alpha=0.5,label=val_set_lbl);

# Set x-axis limits
plt.xlim([epochs[0],epochs[-1]]);

# Title
plt.title(r'Recall vs. Training Epoch $\varepsilon$',
          fontsize=16,weight='bold');

# x-axis label
plt.xlabel(r'Epoch $\varepsilon$',fontsize=14);

# y-axis label
plt.ylabel(r'Recall${}_{\varepsilon}$',fontsize=14);

# Legend
plt.legend();

############
# AUC plot #
############
plt.subplot(5,1,5,sharex=ax1);

# Add grid to axes
plt.grid(color='k',alpha=0.25);

# Cross-entropy error (dB) - training set
plt.plot(epochs,model.history.history['auc'],
    '.-',c='g',alpha=0.5,label=train_set_lbl);

# Cross-entropy error (dB) - validation set
plt.plot(epochs,model.history.history['val_auc'],
    '.-',c='k',alpha=0.5,label=val_set_lbl);

# Set x-axis limits
plt.xlim([epochs[0],epochs[-1]]);

# Title
plt.title(r'Area Under ROC Curve vs. Training Epoch $\varepsilon$',
          fontsize=16,weight='bold');

# x-axis label
plt.xlabel(r'Epoch $\varepsilon$',fontsize=14);

# y-axis label
plt.ylabel(r'AUC${}_{\varepsilon}$',fontsize=14);

# Legend
plt.legend();

# Optimize layout
plt.tight_layout();

### $\textbf{Pass Entire Dataset (i.e., Train }\cup\textbf{ Validation) to Model}$

In [ ]:
# C x 1 class prediction for entire dataset
Y_hat = model.predict(
    X,                        # Input data
    batch_size=N_batch,       # Batch size
);

# Cast 1-hot class labels as ordinal labels
y_hat = argmax(Y_hat,axis=1); # Predictions
y_true = argmax(Y,axis=1);    # Ground truth

### $\textbf{Confusion Matrix Using Predictions Output Above}$

In [ ]:
#########################
#    Compute Matrix:    #
#########################

# Use the sklearn confusion matrix function
confusion_mat = confusion_matrix(y_true,y_hat,
    labels=[c for c in range(C)],sample_weight=None);

#########################
#   Confusion Plotter:  # - adapated from online example
#########################

def plot_confusion_matrix(C,classes,normalize=False,
            title='Confusion matrix',cmap=plt.cm.Greens):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    
    # Let epsilon > 0 be small...
    epsilon = 1e-7;
    
    # Normalization of confusion matrix C
    if normalize:
        C = C.astype('float') / (epsilon + C.sum(axis=1)[:, newaxis]);

    # Print the matrix C
    print(C);
    
    # Image representation
    ax = plt.gca();   # <-- grab current axes
    C = flipud(C);    # <-- flip horizontally for visualization
    im = ax.imshow(C,cmap=cmap);
    
    # Colorbar
    colorbar(im);
    
    # Title
    plt.title(title,fontsize=20,weight="bold");
    
    # Tick labels
    tick_marks = arange(len(classes));
    plt.xticks(tick_marks, classes, rotation=45,fontweight="bold");
    
    # Overwrite default y-axis limits
    plt.ylim([-0.5,len(classes)-0.5]);

    # Text labels in tiles (i.e., elements)
    fmt = '.2f' if normalize else 'd'
    thresh = C.max() / 2.
    for i, j in iter_prod(range(C.shape[0]), range(C.shape[1])):
        plt.text(j, i, format(C[i, j], fmt),
                 horizontalalignment="center",weight="bold",
                 color="white" if C[i, j] > thresh else "black")

    # x-axis labels
    plt.ylabel('True label',fontsize=16,weight="bold");
    
    # y-axis labels
    plt.xlabel('Predicted label',fontsize=16,weight="bold");
    
    # Set axis layout
    plt.tight_layout();
    
#########################
#    Scale Colorbar:    # - found this online
#########################

def colorbar(mappable):
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar


#########################
#    Visualization:     #
#########################

conf_fig = plt.figure(figsize=(9.9,9.9));
plot_confusion_matrix(
    confusion_mat, 
    classes=[clss for clss in class_table],
    normalize=True,
    title='Confusion Matrix',
    cmap=plt.cm.Greens);